In [1]:
import torch
from torch import nn

import torchvision
from torchvision.datasets import ImageFolder

from torchvision import transforms

from torch.utils.data import DataLoader
from pathlib import Path

In [2]:
import sys
sys.path.append("..")

In [3]:
from video_classification.datasets import FolderOfFrameFoldersDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
ROOT = Path("/home/ubuntu/SupervisedVideoClassification")
DATA_ROOT = Path(ROOT/"data")

In [6]:
train_transforms = transforms.Compose([
    torchvision.transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
])

In [7]:
train_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'train', transform=train_transforms)
valid_ds = FolderOfFrameFoldersDataset(DATA_ROOT/'validation', transform=valid_transforms)

In [8]:
train_ds

FolderOfFrameFoldersDataset with 26711 samples.
	Overall data distribution: {'negative': 24747, 'positive': 1964}

In [9]:
valid_ds

FolderOfFrameFoldersDataset with 4751 samples.
	Overall data distribution: {'negative': 4332, 'positive': 419}

In [10]:
from torch import nn
from torchvision.models import resnet101
from video_classification.models.mlp import MLP


class SingleImageResNetModel(nn.Module):
    def __init__(self, mlp_sizes=[768, 128, 2]):
        super().__init__()
        resnet = resnet101(pretrained=True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)

        self.clf = MLP(2048, mlp_sizes)
        self.freeze_resnet()

    def forward(self, x):
        x = self.resnet(x).squeeze()
        x = self.clf(x)
        return x

    def freeze_resnet(self):
        for p in self.resnet.parameters():
            p.requires_grad = False

    def unfreeze_resnet(self):
        for p in self.resnet.parameters():
            p.requires_grad = True


In [11]:
model = SingleImageResNetModel(mlp_sizes=[1024, 256, 2])

model = model.to(device)

In [12]:
from video_classification.trainer import Trainer

classes_weights = torch.Tensor([0.3, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=classes_weights)

In [13]:
trainer = Trainer(train_ds, 
                  valid_ds, 
                  model, 
                  criterion,
                  "single_frame_resnet",
                  str(ROOT/'checkpoints'),
                  device=device,
                  amp_opt_level="O1",
                  cycle_mult=0.9,
                 )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [14]:
trainer.train(lr=1e-3, 
              batch_size=128, 
              n_epochs=20,
              gradient_accumulation_steps=2,
              num_workers=8,
              max_gradient_norm=2.0,
             )

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Training Results - Epoch: 1: Avg accuracy: 0.95 |Precision: 0.95, 0.88 |Recall: 1.00, 0.35 | F1: 0.74 | Avg loss: 0.34
Validation Results - Epoch: 1: Avg accuracy: 0.89 |Precision: 0.93, 0.37 |Recall: 0.95, 0.30 | F1: 0.64 | Avg loss: 0.46


Training Results - Epoch: 2: Avg accuracy: 0.88 |Precision: 0.98, 0.34 |Recall: 0.89, 0.74 | F1: 0.70 | Avg loss: 0.34
Validation Results - Epoch: 2: Avg accuracy: 0.79 |Precision: 0.94, 0.21 |Recall: 0.82, 0.50 | F1: 0.59 | Avg loss: 0.52


Training Results - Epoch: 3: Avg accuracy: 0.85 |Precision: 0.98, 0.29 |Recall: 0.86, 0.73 | F1: 0.67 | Avg loss: 0.36
Validation Results - Epoch: 3: Avg accuracy: 0.75 |Precision: 0.94, 0.17 |Recall: 0.78, 0.47 | F1: 0.55 | Avg loss: 0.55


Training Results - Epoch: 4: Avg accuracy: 0.95 |Precision: 0.97, 0.74 |Recall: 0.98, 0.57 | F1: 0.81 | Avg loss: 0.24
Validation Results - Epoch: 4: Avg accuracy: 0.89 |Precision: 0.94, 0.37 |Recall: 0.94, 0.33 | F1: 0.64 | Avg loss: 0.44


...

Training Results - Epoch: 16: Avg accuracy: 0.96 |Precision: 0.98, 0.72 |Recall: 0.98, 0.77 | F1: 0.86 | Avg loss: 0.16
Validation Results - Epoch: 16: Avg accuracy: 0.91 |Precision: 0.95, 0.49 |Recall: 0.95, 0.47 | F1: 0.71 | Avg loss: 0.40


Training Results - Epoch: 17: Avg accuracy: 0.96 |Precision: 0.98, 0.76 |Recall: 0.98, 0.76 | F1: 0.87 | Avg loss: 0.17
Validation Results - Epoch: 17: Avg accuracy: 0.91 |Precision: 0.94, 0.52 |Recall: 0.97, 0.36 | F1: 0.69 | Avg loss: 0.47


Training Results - Epoch: 18: Avg accuracy: 0.97 |Precision: 0.98, 0.80 |Recall: 0.99, 0.71 | F1: 0.87 | Avg loss: 0.17
Validation Results - Epoch: 18: Avg accuracy: 0.92 |Precision: 0.94, 0.59 |Recall: 0.97, 0.38 | F1: 0.71 | Avg loss: 0.45


Training Results - Epoch: 19: Avg accuracy: 0.96 |Precision: 0.98, 0.72 |Recall: 0.98, 0.80 | F1: 0.87 | Avg loss: 0.15
Validation Results - Epoch: 19: Avg accuracy: 0.91 |Precision: 0.94, 0.51 |Recall: 0.96, 0.41 | F1: 0.70 | Avg loss: 0.45


Training Results - Epoch: 20: Avg accuracy: 0.97 |Precision: 0.98, 0.79 |Recall: 0.98, 0.76 | F1: 0.88 | Avg loss: 0.16
Validation Results - Epoch: 20: Avg accuracy: 0.93 |Precision: 0.95, 0.72 |Recall: 0.98, 0.42 | F1: 0.75 | Avg loss: 0.44


In [15]:
import pandas as pd
reform = {(outerKey, innerKey): values for outerKey, innerDict in trainer.epoch_state.items() for innerKey, values in innerDict.items()}
pd.DataFrame(reform).T

accuracy        f1       nll  \
1  train  0.948935    0.7389    0.3364   
   test   0.894338  0.636654  0.460383   
2  train   0.87698  0.699279  0.336004   
   test   0.790149  0.586061  0.521678   
3  train  0.849163  0.665361  0.361623   
   test   0.751842  0.551804  0.553447   
4  train   0.95354  0.808503  0.244142   
   test   0.890339  0.644035  0.435605   
5  train  0.940998  0.802461  0.233504   
   test   0.840244  0.625963  0.466015   
6  train  0.947699  0.809268  0.243121   
   test   0.892444  0.660857   0.45323   
7  train  0.959904  0.811412  0.251877   
   test   0.936224  0.706117  0.477989   
8  train   0.93411  0.800068  0.220587   
   test   0.853505  0.645018  0.450793   
9  train  0.954738  0.837576  0.197617   
   test    0.90844  0.673566  0.432827   
10 train  0.947999  0.828947  0.191406   
   test   0.896443  0.672389  0.433336   
11 train  0.962562  0.843118  0.209256   
   test   0.929489   0.71733  0.431287   
12 train  0.961589  0.851849  0.182364   
   test   0.931172  0.749499  0.366148   
13 train  0.963386  0.864034  0.175074   
   test   0.913071  0.698497  0.433779   
14 train  0.949459  0.833029  0.187926   
   test   0.883393  0.695248   0.38541   
15 train  0.956797  0.852438  0.165108   
   test    0.91265  0.688575   0.45094   
16 train  0.961589  0.863005  0.163038   
   test   0.909282  0.712691  0.397953   
17 train  0.964621  0.870194  0.170055   
   test   0.914123  0.690275  0.474155   
18 train   0.96552  0.866446  0.171757   
   test   0.921701  0.708548  0.452595   
19 train  0.962675  0.869906  0.149125   
   test   0.913702  0.703243  0.454537   
20 train  0.967878  0.879744  0.155042   
   test   0.933909   0.74508  0.435927   

                                          precision  \
1  train    [0.9510435554183866, 0.879746835443038]   
   test   [0.9332579185520362, 0.37462235649546827]   
2  train   [0.9769747077388096, 0.3431419079259611]   
   test   [0.9440745672436751, 0.20983935742971888]   
3  train  [0.9760569852941177, 0.29145627146031106]   
   test   [0.9387698302254384, 0.17184801381692574]   
4  train   [0.9662038873462911, 0.7408394403730846]   
   test   [0.9359414321665522, 0.36578947368421055]   
5  train   [0.9761229605885012, 0.5815811606391926]   
   test   [0.9449564134495642, 0.26902173913043476]   
6  train   [0.9722896440129449, 0.6423907584128579]   
   test    [0.9397008055235904, 0.3866995073891626]   
7  train   [0.9612327592924491, 0.9272727272727272]   
   test              [0.9357560025957171, 0.953125]   
8  train    [0.982211034193413, 0.5354659248956884]   
   test    [0.9477832512315271, 0.2995658465991317]   
9  train    [0.9774442190669371, 0.683163512857836]   
   test    [0.9379635873229939, 0.4735099337748344]   
10 train   [0.9816479709666777, 0.6167275680064962]   
   test    [0.9415823367065317, 0.4094292803970223]   
11 train   [0.9698468600371968, 0.8347222222222223]   
   test     [0.940683572216097, 0.6944444444444444]   
12 train   [0.9753135895483509, 0.7667804323094426]   
   test     [0.948488241881299, 0.6608391608391608]   
13 train   [0.9792716860910594, 0.7575757575757576]   
   test    [0.9426248023492206, 0.5092592592592593]   
14 train   [0.9820346944661914, 0.6257166257166257]   
   test   [0.9549614643545279, 0.38731218697829717]   
15 train   [0.9829287264093012, 0.6773204903677759]   
   test    [0.9402112834344797, 0.5066225165562914]   
16 train   [0.9816837915769808, 0.7246168582375478]   
   test   [0.9484939066452058, 0.48507462686567165]   
17 train   [0.9809262102966136, 0.7592875318066158]   
   test    [0.9401076716016151, 0.5187713310580204]   
18 train   [0.9770925110132158, 0.7995404939689833]   
   test     [0.9417670682730924, 0.587360594795539]   
19 train   [0.9842985318107668, 0.7206754906435417]   
   test     [0.9436651583710407, 0.513595166163142]   
20 train    [0.981031774797632, 0.7941489361702128]   
   test    [0.9456521739130435, 0.7160493827160493]   

                               